**SQLite Database**

In [ ]:
import sqlite3

In [ ]:
# create a connection to SQLite DB
conn = sqlite3.connect("sample.db")

In [ ]:
# create a cursor to help us execute the SQL commands
cursor = conn.cursor()

In [ ]:
# create a table
# stocks : stock_code, stock_name

cursor.execute(
"""
CREATE TABLE IF NOT EXISTS stocks (
  stock_code INTEGER PRIMARY KEY,
  stock_name TEXT NOT NULL
)
"""
)


In [ ]:
# insert some data
cursor.execute(
    "INSERT INTO stocks (stock_name) VALUES (?)", ('TESLA',)
)



In [ ]:
cursor.execute(
    "INSERT INTO stocks (stock_name) VALUES (?)", ('Microsoft',)
)


In [ ]:
# select records

cursor.execute("SELECT * FROM stocks")

In [ ]:
rows = cursor.fetchall()
rows

[(1, 'TESLA'), (2, 'Microsoft')]

In [ ]:
conn.commit()

In [ ]:
conn.close()

**Using SQLite as vector storage**

In [ ]:
# vector = array of numbers -> numpy arrays -> [1.2, 2,5, 3,7]
# store the information into a bytes format

In [ ]:
import numpy as np

In [ ]:
conn = sqlite3.connect("sample_vectors.db")

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute(
"""
CREATE TABLE IF NOT EXISTS vectors (
  vector_id INTEGER PRIMARY KEY,
  vector BLOB NOT NULL
)
"""
)

In [ ]:
vect_tsla = np.array([1.3, 3.5, 2.2, 0.9])
vect_msft = np.array([2.8, 1.6, 3.8, 2.2])

In [ ]:
type(vect_tsla)

numpy.ndarray

In [ ]:
vect_tsla.tobytes()

b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'

In [ ]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect_tsla.tobytes()),))

In [ ]:
cursor.execute("INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect_msft.tobytes()),))

In [ ]:
cursor.execute("SELECT * FROM vectors")

In [ ]:
rows = cursor.fetchall()
rows

[(1,
  b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'),
 (2,
  b'ffffff\x06@\x9a\x99\x99\x99\x99\x99\xf9?ffffff\x0e@\x9a\x99\x99\x99\x99\x99\x01@')]

In [ ]:
# Deserialization

In [ ]:
rows

[(1,
  b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?'),
 (2,
  b'ffffff\x06@\x9a\x99\x99\x99\x99\x99\xf9?ffffff\x0e@\x9a\x99\x99\x99\x99\x99\x01@')]

In [ ]:
# np.frombuffer

In [ ]:
v = np.frombuffer(rows[0][1], dtype=np.float64)
v

array([1.3, 3.5, 2.2, 0.9])

In [ ]:
vectors = []
for row in rows:
  v = np.frombuffer(row[1], dtype=np.float64)
  vectors.append(v)

In [ ]:
vectors

[array([1.3, 3.5, 2.2, 0.9]), array([2.8, 1.6, 3.8, 2.2])]

In [ ]:
# Find nearest vector

In [ ]:
q_vector = np.array([3.5, 2.2, 3.5, 5.5])

In [ ]:
cursor.execute("""
SELECT vector FROM vectors ORDER BY abs(vector - ?) DESC
""", (sqlite3.Binary(q_vector.tobytes()),))

In [ ]:
res = cursor.fetchone()

In [ ]:
res

(b'\xcd\xcc\xcc\xcc\xcc\xcc\xf4?\x00\x00\x00\x00\x00\x00\x0c@\x9a\x99\x99\x99\x99\x99\x01@\xcd\xcc\xcc\xcc\xcc\xcc\xec?',)

In [ ]:
np.frombuffer(res[0], dtype=np.float64)

array([1.3, 3.5, 2.2, 0.9])